In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
Key = os.getenv("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(openai_api_key = Key, model_name = "gpt-3.5-turbo", temperature = 0.5)

C:\Users\user\AppData\Local\Temp\ipykernel_2988\3061724652.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key = Key, model_name = "gpt-3.5-turbo", temperature = 0.5)


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import  get_openai_callback
import PyPDF2


In [7]:


TEMPLATE ="""
Text:{text}
You are an expert in Python programming. Generate {num_questions} multiple-choice questions based on the {topic} provided and answer them using the given {text}.
Each question should have 4 options (A,B,C,D) and one correct answer.
The questions should be of {difficulty} difficulty level.
Generate the questions in the following format:

Question:
(Write the question here)

Options:
A) Option 1
B) Option 2
C) Option 3
D) Option 4

Correct Answer:
(Write the correct option letter here, e.g., “B”)

Explanation:
(Briefly explain why the correct option is right and why others are wrong — optional but recommended.)
##RESPONSE_FORMAT##
{response_json}


##Note: Don't forget to ask the mentioned number of questions in the response.
"""


In [8]:
RESPONSE_JSON = """Question:
(Write the question here)

Options:
A) Option 1
B) Option 2
C) Option 3
D) Option 4

Correct Answer:
(Write the correct option letter here, e.g., “B”)

Explanation:
(Briefly explain why the correct option is right and why others are wrong — optional but recommended.)"""

In [9]:
quiz_generation_prompt = PromptTemplate(input_variables=["topic", "difficulty", "num_questions","text","response_json"],
                                        template=TEMPLATE)


In [10]:
chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

C:\Users\user\AppData\Local\Temp\ipykernel_2988\1736729569.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)


In [11]:
TEMPLATE2 = """
You are an expert {topic} reviewer and I need you to {num_questions} and review all of them. Review and analyze the following multiple-choice questions generated by an AI model.
Evaluate the quality of the questions, options, and answers. Provide feedback on clarity, relevance, and correctness from the {quiz}.
"""

In [12]:
quiz_review_prompt = PromptTemplate(input_variables=["quiz","topic"], template=TEMPLATE2)

In [13]:
review_chain = LLMChain(llm=llm, prompt=quiz_review_prompt, output_key="review", verbose=True)

In [14]:
generate_quiz_chain = SequentialChain(
    chains=[chain, review_chain],
    input_variables=["topic", "difficulty", "num_questions", "text","response_json"],
    output_variables=["quiz", "review"],
    verbose=True
    )

In [15]:
file_path = r"C:\Users\user\raghavendra\genai_project\data.txt"


In [16]:
with open(file_path, "r") as file:
    text = file.read()

In [17]:
print(text)

Python is a popular programming language. It was created by Guido van Rossum, and released in 1991.

It is used for:

web development (server-side),
software development,
mathematics,
system scripting.
What can Python do?
Python can be used on a server to create web applications.
Python can be used alongside software to create workflows.
Python can connect to database systems. It can also read and modify files.
Python can be used to handle big data and perform complex mathematics.
Python can be used for rapid prototyping, or for production-ready software development.
Why Python?
Python works on different platforms (Windows, Mac, Linux, Raspberry Pi, etc).
Python has a simple syntax similar to the English language.
Python has syntax that allows developers to write programs with fewer lines than some other programming languages.
Python runs on an interpreter system, meaning that code can be executed as soon as it is written. This means that prototyping can be very quick.
Python can be tr

In [18]:
topic = "Python Programming"
difficulty = "easy"
num_questions = 10

In [19]:
with get_openai_callback() as cb:

    response = generate_quiz_chain(
    {
        "topic":topic,
        "difficulty":difficulty,
        "num_questions":5,
        "text":text,
        "response_json":json.dumps(RESPONSE_JSON)  # Ensure response_json is a valid JSON string
    }
    )
    print("Total Tokens Used:", cb.total_tokens)
    print("Total Cost: $", cb.total_cost)
    
     

C:\Users\user\AppData\Local\Temp\ipykernel_2988\475275216.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_quiz_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Python is a popular programming language. It was created by Guido van Rossum, and released in 1991.

It is used for:

web development (server-side),
software development,
mathematics,
system scripting.
What can Python do?
Python can be used on a server to create web applications.
Python can be used alongside software to create workflows.
Python can connect to database systems. It can also read and modify files.
Python can be used to handle big data and perform complex mathematics.
Python can be used for rapid prototyping, or for production-ready software development.
Why Python?
Python works on different platforms (Windows, Mac, Linux, Raspberry Pi, etc).
Python has a simple syntax similar to the English language.
Python has syntax that allows developers to write programs with fewer lines than some other programming languages.
Python runs on an interpreter system, meaning that co

In [20]:
quiz = response.get("quiz")

In [21]:
print(quiz)

Question:
What is Python primarily used for?

Options:
A) Web development
B) Mobile app development
C) Game development
D) Graphic design

Correct Answer:
A) Web development

Explanation:
Python is commonly used for web development, including server-side programming, due to its versatility and ease of use in this domain.

Question:
Which version of Python is currently recommended for use in this tutorial?

Options:
A) Python 1
B) Python 2
C) Python 3
D) Python 4

Correct Answer:
C) Python 3

Explanation:
Python 3 is the most recent major version of Python and is the recommended version for use in most cases due to its ongoing updates and improvements.

Question:
How does Python define scope for loops, functions, and classes?

Options:
A) Using curly brackets
B) Using indentation and whitespace
C) Using semicolons
D) Using parentheses

Correct Answer:
B) Using indentation and whitespace

Explanation:
Python relies on indentation to define scope, which is different from many other progra

In [24]:
json_data = json.dumps(RESPONSE_JSON)

In [25]:
json_data 

'"Question:\\n(Write the question here)\\n\\nOptions:\\nA) Option 1\\nB) Option 2\\nC) Option 3\\nD) Option 4\\n\\nCorrect Answer:\\n(Write the correct option letter here, e.g., \\u201cB\\u201d)\\n\\nExplanation:\\n(Briefly explain why the correct option is right and why others are wrong \\u2014 optional but recommended.)"'